Workings with elective data

In [ ]:
%cd ../src

In [ ]:
from pathlib import Path
from typing import List

import pandas as pd
from pydantic import BaseModel

Make sure you are simulating work as if you're in the src directory

In [ ]:
from mock import mock

In [ ]:
electives_file = mock.path_to_hdf_file("electives")
df = pd.read_hdf(electives_file)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.to_csv("tmp.csv")

In [ ]:
engine = mock.make_mock_db_in_memory("electives")

In [ ]:
import sqlalchemy as sa

from api.electives import QUERY_MOCK_PATH

In [ ]:
QUERY_MOCK_PATH.read_text()

In [ ]:
# with engine.connect() as conn:
#     _ = pd.read_sql("select * from electives", conn)
#     _.head()